In [57]:
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint
from langchain_core.tools import tool,Tool
from dotenv import load_dotenv
load_dotenv()
import requests

In [58]:
from langchain_community.tools import DuckDuckGoSearchRun,DuckDuckGoSearchResults
search_run = DuckDuckGoSearchRun()
search_tool = Tool(
    name="DuckDuckGo Search",
    func=search_run.run,   # use .run, not invoke
    description="Use this tool to get current news or web search results."
)
result=search_tool.invoke("top news in india today")

In [77]:
@tool
def get_weather_data(city: str)-> str:
    """this funciton fetches the correct weather data for a given city"""
    url = f'https://api.weatherstack.com/current?access_key=4d1d8ae207a8c845a52df8a67bf3623e&query={city}'

    response = requests.get(url)

    return response.json()   


In [59]:
result

"Check today's top 10 news stories, headline news, breaking news, latest news, politics news, sports news, entertainment news and business news on Times of India Check out the latest news from India and around the world. Latest India news on Bollywood, Politics, Business, Cricket, Technology and Travel. 5 days ago · Today's top India news headlines, news on Indian politics, elections, government, business, technology, and Bollywood. Read full articles, watch videos, browse thousands of titles and more on the 'India' topic with Google News. 2 days ago · News Today Live Updates: Get the latest news updates from India and all around the world. Stay updated with breaking news today. Top News Headlines of October 19, 2025 | In the …"

In [81]:
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    task="text-generation",
    temperature=0.5,
    max_new_tokens=512
)

model=ChatHuggingFace(llm=llm)

In [82]:
from langchain.agents import create_react_agent,AgentExecutor
from langchain import hub

In [83]:
#step 2: Pull the REACT prompt from Langchain Hub 
prompt=hub.pull("hwchase17/react")

In [86]:
# Step 3: Create the ReAct agent manually with the pulled prompt
agent=create_react_agent(
    llm=model,
    tools=[search_tool,get_weather_data],
    prompt=prompt
)

In [87]:
#step 4: wrap it with AgentExecutor

agent_executor=AgentExecutor(
    agent=agent,
    tools=[search_tool,get_weather_data],
    verbose=True,
    handle_parsing_errors=True
)

In [88]:
#step 5: Invoke

response=agent_executor.invoke({"input":"Find the capital city of Karnataka then find its current weather condition"})
print(response)



> Entering new AgentExecutor chain...
 I need to find the capital city of Karnataka and then get the current weather condition for that city.

Action: DuckDuckGo Search
Action Input: capital city of Karnataka
Bengaluru Bengaluru, also known as Bangalore, is the capital city of Karnataka and one of India's most dynamic metropolitan hubs. Renowned as the "Silicon Valley of India", it is a global technology and innovation center, home to leading IT companies, startups, and research institutions. Learn about the origins and evolution of Bengaluru, the third most populous city and the capital of Karnataka state in India. Discover its cultural, political and economic significance as a hub of IT, education and aerospace. Nestled in the heart of southern India, Mysore (now officially Mysuru) is a city where history, heritage, and modern charm converge. Known as the Cultural Capital of Karnataka , Mysore is famed for its palaces, yoga, silk sarees, and rich traditions. Whether you're a histor

In [89]:
response['output']

'The capital city of Karnataka, Bengaluru, is currently experiencing unsettled weather conditions with rain and thunderstorms.'